Before understanding the yield keyword, we must understand what generators are. And before generators come iterables.

#### Iterables

When you create a list, you read its items one by one. Reading its items one by one is called iteration


In [3]:
mylist = [1,2,3]
for i in mylist:
    print(i)

1
2
3


mylist is an iterable. When you use list comprehension, you create a list, and so an iterable

In [4]:
mlist = [x*x for x in range(3)]
for i in mylist:
    print(i)

1
2
3


Everything you can use 'for...in...'on is an iterable: list, strings files etc. That is anything that can be 'iterated on'. 

These iterables are handy because you can read them as much as you wish, but you store all the values in memory and this is not alwys what you want when you have a lot of values.

#### Generators

Generators are iterators, a kind of iterable you can only iterate over once. Generators do not store all the values in memory, they generate the values on the fly.


In [6]:
mygenerator = (x*x for x in range(3))
for i in mygenerator:
    print(i)


0
1
4


It is just the same except you used () instead of []. BUT, you cannot perform for i in mygenerator a second time since generators can only be used once: they calculate 0, then forget about it and calculate 1, and end calculating 4, one by one

#### Yield

Yield is a keyword that is used like return, except the function will return a generator. 

In [8]:
def createGenerator():
    mylist = range(3)
    for i in mylist:
        yield i*i

mygenerator = createGenerator() # create a generator
print(mygenerator) # generator is an object

for i in mygenerator:
    print(i)
    

<generator object createGenerator at 0x0000029AEC016410>
0
1
4


Here it's a useless example, but it's handy when you know your function will return a huge set of values that you will only need to read once.

To master yield, you must understand that when you call the function, the code you have written in the function body does not run. The function only returns the generator object, this is a bit tricky :-)

Then, your code will continue from where it left off each time for uses the generator.

Now the hard part:

The first time the for calls the generator object created from your function, it will run the code in your function from the beginning until it hits yield, then it'll return the first value of the loop. Then, each other call will run the loop you have written in the function one more time, and return the next value, until there is no value to return.

The generator is considered empty once the function runs, but does not hit yield anymore. It can be because the loop had come to an end, or because you do not satisfy an "if/else" anymore.

The yield keyword is reduced to two simple facts:

1. If the compiler detects the yield keyword anywhere inside a function, that function no longer returns via the return statement. Instead, it immediately returns a lazy "pending list" object called a generator
2. A generator is iterable. What is an iterable? It's anything like a list or set or range or dict-view, with a built-in protocol for visiting each element in a certain order.

In a nutshell: a generator is a lazy, incrementally-pending list, and yield statements allow you to use function notation to program the list values the generator should incrementally spit out.

##### Example

Consider this example of classic fibbonaci.

we pass a limit and it will compute all the fibonaci numbers upto that limit


In [10]:
def classic_fibonacci(limit):
    nums = []
    current, nxt=0, 1
    while current < limit:
        current, nxt = nxt, nxt+current
        nums.append(current)
        
    return nums

In [25]:
def generator_fibonacci():
    current, nxt=0, 1
    
    while True:
        current, nxt = nxt, nxt+current
        yield current
        
#generaotrs are composible
def even_genertor(numbers):
    for n in numbers:
        if n % 2 == 0:
            yield n

##consume both generators as a pipeline here
def even_fib():
    for n in even_genertor(generator_fibonacci()):
        yield n
        

In [28]:
if __name__ == '__main__':
    print("Classic")
    for m in classic_fibonacci(100):
        print(m,end=', ')
    print()
    
    print("generator")
    for m in generator_fibonacci():
        print(m,end=', ')
        if m >100:
            break
    print()
    print("composed")
    for m in even_fib():
        print(m,end=', ')
        if m >1000000:
            break
    print()

Classic
1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 
generator
1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 
composed
2, 8, 34, 144, 610, 2584, 10946, 46368, 196418, 832040, 3524578, 


The list nums[] is where the function does all the work and stores the numbers. what if you need the first million fibonaci..how long will the function takes? what if you need to look for number satisify some property? what will be the memory consumption 

This can be done with an on-demand high performance way